# 13. Naive Bayes

Uma rede social não é tão boa se as pessoas não conseguem se conectar. Portanto, a DataSciencester possui um atributo popular que permite que membros enviem mensagens uns aos outros. E enquanto a maioria dos membros são cidadãos responsáveis que somente enviam mensagens de “como você está?”, alguns são canalhas e enviam mensagens de spam sobre esquemas para ficarem ricos, medicamentos sem receita e programas de credenciamento de data science. Seus usuários começaram a reclamar e a Vice-presidente de Mensagem pediu que você usasse data science para descobrir como filtrar essas mensagens de spam.

### Um Filtro de Spam Muito Estúpido

* Imagine um "universo" que consiste em receber uma mensagem escolhida ao acaso entre todas as possíveis.
  * S: a mensagem é spam.
  * V: A mensagem contém a palavra "viagra".
  
  
* O teorema de Bayes (Cap. 06) diz que a probabilidade de uma mensagem ser spam depender da palavra "viagra" é dado por:

   $P(S|V) = [P(V|S)*P(S)] / [P(V|S)*P(S)+P(V|not(S))*P(not(S))]$

* Esse cálculo pode ser visto como a proporção de mensagens com a palavra "viagra" que são spam.
* Se temos uma grande quantidade de mensagens que sabemos que não são spam e outra grande qauntidade que sabemos ser spam, então é possível calcular $P(V|S)$ e $P(V|not(S))$.
* Se presumimos que uma mensagem possui igual probabilidade de ser spam ou não spam ($P(S) = P(not(S)) = 0.5$), então:

   $P(S|V) = P(V|S)/[P(V|S)+P(V|not(S))]$

* Ex.: 50% das mensagens de spam possuem a palavra "viagra". 1% das mensagens não spam possuem. A probabilidade de que qualquer email que tenha a palavra "viagra" seja spam é:

   $0.5/(0.5+0.01) = 98\%$

### Um Filtro de Spam Mais Sofisticado

* Imagine que temos um vocabulário de muitas palavras $w_{1}$, ..., $w_{n}$.
* Usaremos $X_{i}$ para o evento "a mensagem contém a palavra $w_{i}$".
* Imagine também que encontramos um $P(X_{i}|S)$ como estimativa da probabilidade de a mensagem de spam conter a palavra $w_{i}$ e $P(X_{i}|not(S))$ de a mensagem de não spam conter a palavra $w_{i}$.

* A chave para naive bayes é fazer a suposição de que as presenças (ou ausências) de cada palavra são independentes uma das outras, condição para uma mensagem ser spam ou não.
  * Significa que a presença de uma palavra na mensagem não indica a presença de outra. Matematicamente:
  
    $P(X_{1} = x_{1}, ..., X_{n} = x_{n}|S) = P(X_{1} = x_{1}|S) * ... * P(X_{n} = x_{n}|S)$
    
* É uma hipótese extrema.
* Imagine que todo o nosso vocabulário consista apenas das palavras “viagra” e “rolex”, e que metade de todas as mensagens de spam sejam “viagra barato” e que a outra metade seja “rolex autêntico”.
* A estimativa que um spam conhtenha os dois é:

   $P(X_{1} = 1, X_{2} = 1|S) = P(X_{1} = 1|S)*P(X_{2} = 1|S) = 0.5*0.5 = 0.25$
   
* assim, afastamos a teoria de que "viagra" e "rolex" não apareçam juntos.
* O Teorema de Bayes também diz que podemos calcular a probabilidade de uma mensagem ser spam usando a equação:

   $P(S|X = x) = P(X = x|S)/[P(X = x|S)+P(X = x|not(S))]$